# Semantic Kernel Tool Use Example

This document provides an overview and explanation of the code used to create a Semantic Kernel-based tool that integrates with Azure AI Search for Retrieval-Augmented Generation (RAG). The example demonstrates how to build an AI agent that retrieves travel documents from an Azure AI Search index, augments user queries with semantic search results, and streams detailed travel recommendations.

## Initializing the Environment

### Importing Packages
The following code imports the necessary packages:

In [1]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField

from openai import AsyncOpenAI

from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.agents import ChatCompletionAgent

### Creating the Semantic Kernel and AI Service

A Semantic Kernel instance is created and configured with an asynchronous OpenAI chat completion service. The service is added to the kernel for use in generating responses.

In [2]:
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)

# Create a Semantic Kernel instance and add an OpenAI chat completion service.
kernel = Kernel()
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
    service_id="agent",
)
kernel.add_service(chat_completion_service)

### Defining the Prompt Plugin

The PromptPlugin is a native plugin that defines a function to build an augmented prompt using retrieval context

In [3]:
class PromptPlugin:
    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context."
    )
    @staticmethod
    def build_augmented_prompt(query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide your answer."
        )

# Register the plugin with the kernel.
kernel.add_plugin(PromptPlugin(), plugin_name="promptPlugin")

KernelPlugin(name='promptPlugin', description=None, functions={})

## Vector Database Initialization

We initialize Azure AI Search with persistent storage and add enhanced sample documents. Azure AI Search will be used to store and retrieve documents that provide context for generating accurate responses.

In [ ]:
# Initialize Azure AI Search with persistent storage
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = "travel-documents"

search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# Define the index schema
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String)
]

index = SearchIndex(name=index_name, fields=fields)

# Create the index
index_client.create_index(index)

# Enhanced sample documents
documents = [
    {"id": "1", "content": "Contoso Travel offers luxury vacation packages to exotic destinations worldwide."},
    {"id": "2", "content": "Our premium travel services include personalized itinerary planning and 24/7 concierge support."},
    {"id": "3", "content": "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage."},
    {"id": "4", "content": "Popular destinations include the Maldives, Swiss Alps, and African safaris."},
    {"id": "5", "content": "Contoso Travel provides exclusive access to boutique hotels and private guided tours."}
]

# Add documents to the index
search_client.upload_documents(documents)


A helper function `get_retrieval_context` is defined to query the index and return the top two relevant documents based on the user query:

In [7]:
def get_retrieval_context(query: str) -> str:
    results = search_client.search(query)
    context_strings = []
    for result in results:
        context_strings.append(f"Document: {result['content']}")
    return "\n\n".join(context_strings) if context_strings else "No results found"

## Setting the Function Choice Behavior 

In Semantic Kernel, we have the ability to have some control of the agent choice of functions. This is done by using the `FunctionChoiceBehavior` class. 

The code below sets it to `Auto` which allows the agent to choose among the available functions or not choose any. 

This can also be set to:
`FunctionChoiceBehavior.Required` - to require the agent to choose at least one function 
`FunctionChoiceBehavior.NoneInvoke` - instructs the agent to not choose any function. (good for testing)

In [8]:
settings = kernel.get_prompt_execution_settings_from_service_id("agent")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
arguments = KernelArguments(settings=settings)

In [9]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = (
    "Answer travel queries using the provided context. If context is provided, do not say 'I have no context for that.'"
)
agent = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=arguments,
)

A helper function `get_augmented_prompt` forces a call to the plugin to build the augmented prompt. It directly calls the static plugin method:

In [10]:
async def get_augmented_prompt(query: str) -> str:
    retrieval_context = get_retrieval_context(query)
    return PromptPlugin.build_augmented_prompt(query, retrieval_context)

### Running the Agent with Streaming Chat History
The main asynchronous loop creates a chat history for the conversation and, for each user input, first adds the augmented prompt (as a system message) to the chat history so that the agent sees the retrieval context. The user message is also added, and then the agent is invoked using streaming. The output is printed as it streams in.

In [11]:
async def main():
    # Create a chat history.
    chat_history = ChatHistory()
    
    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?", # Retrieval context available.
        "What is Neural Network?" # No retrieval context available.
    ]
    
    for user_input in user_inputs:
        # Obtain the augmented prompt.
        augmented_prompt = await get_augmented_prompt(user_input)
        # Add the augmented prompt as a system message so the agent sees the retrieval context.
        chat_history.add_system_message(augmented_prompt)
        # Also add the user message.
        chat_history.add_user_message(user_input)
        
        print(f"# User: '{user_input}'")
        
        agent_name: str | None = None
        print("# Assistant - ", end="")
        async for content in agent.invoke_stream(chat_history):
            if not agent_name:
                agent_name = content.name
                print(f"{agent_name}: '", end="")
            if (
                not any(isinstance(item, (FunctionCallContent, FunctionResultContent))
                        for item in content.items)
                and content.content.strip()
            ):
                print(f"{content.content}", end="", flush=True)
        print("\n" + "="*60 + "\n")
        
        # Clear chat history for the next query.
        chat_history = ChatHistory()

await main()

# User: 'Can you explain Contoso's travel insurance coverage?'
# Assistant - TravelAgent: 'Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage. This means that if you encounter a medical issue while traveling, have to cancel your trip for unforeseen reasons, or lose your luggage, you are protected under their insurance plan.

# User: 'What is Neural Network?'
# Assistant - TravelAgent: 'I have no context for that.

